In [1]:
# 初始化环境
import os

os.environ['OPENAI_BASE_URL'] = 'https://key.wenwen-ai.com/v1'
os.environ['OPENAI_API_KEY'] = 'sk-6V2exWFBSa2lmuZ7C0D773D1BaEd4fB7A1B6A0A265D550C6'

In [2]:
# 一般参数
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-4-turbo-preview",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello! "}
    ],
    n=1,
    max_tokens=2048,
    temperature=0.8,
    # top_p=1,
    # stop="!",
    # seed=1234567,
    # user="user_1234567",
)

print(completion.model_dump_json())

{"id": "chatcmpl-8lI0hdhhCWbcrhJ2z3OLxbq4hb56u", "choices": [{"finish_reason": "stop", "index": 0, "message": {"content": "Hello! How can I assist you today?", "role": "assistant", "function_call": null, "tool_calls": null}, "logprobs": null}], "created": 1706281339, "model": "gpt-4-0125-preview", "object": "chat.completion", "system_fingerprint": "fp_376b7f78b9", "usage": {"completion_tokens": 9, "prompt_tokens": 20, "total_tokens": 29}}


In [14]:
# system message 一般用作一个整体的
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "你是一个python专家，你的名字叫小问"},
        # {"role": "user", "content": "你是一个python专家，你的名字叫小问"},
        {"role": "user", "content": "你的名字是什么？如何入门学习python"}
    ],
    n=1,
    max_tokens=2048,
    temperature=0.8,
)

print(completion.model_dump_json())

{"id": "chatcmpl-8jQRQLehUtzusg7vkdONa6hnUuZEv", "choices": [{"finish_reason": "stop", "index": 0, "message": {"content": "\u4f60\u597d\uff0c\u6211\u7684\u540d\u5b57\u662f\u5c0f\u95ee\u3002\u8981\u5165\u95e8\u5b66\u4e60Python\uff0c\u4f60\u53ef\u4ee5\u6309\u7167\u4ee5\u4e0b\u6b65\u9aa4\u8fdb\u884c\uff1a\n\n1. \u5b89\u88c5Python\uff1a\u9996\u5148\uff0c\u4f60\u9700\u8981\u5728\u4f60\u7684\u8ba1\u7b97\u673a\u4e0a\u5b89\u88c5Python\u3002\u4f60\u53ef\u4ee5\u4ecePython\u5b98\u65b9\u7f51\u7ad9\u4e0a\u4e0b\u8f7d\u9002\u5408\u4f60\u64cd\u4f5c\u7cfb\u7edf\u7684\u5b89\u88c5\u7a0b\u5e8f\uff0c\u5e76\u6309\u7167\u6307\u5bfc\u8fdb\u884c\u5b89\u88c5\u3002\n\n2. \u5b66\u4e60\u57fa\u7840\u77e5\u8bc6\uff1a\u4e00\u65e6\u5b89\u88c5\u5b8c\u6210\uff0c\u4f60\u53ef\u4ee5\u5f00\u59cb\u5b66\u4e60Python\u7684\u57fa\u7840\u77e5\u8bc6\uff0c\u6bd4\u5982\u53d8\u91cf\u3001\u6570\u636e\u7c7b\u578b\u3001\u6761\u4ef6\u8bed\u53e5\u3001\u5faa\u73af\u7b49\u3002\u4f60\u53ef\u4ee5\u901a\u8fc7\u9605\u8bfbPython\u7684\u5b98\u65b

In [19]:
# response_format 参数 https://platform.openai.com/docs/guides/text-generation/json-mode
from openai import OpenAI
from openai.types.chat import completion_create_params

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": "Who won the world series in 2020?"}
    ],
    # 对content进行json格式化
    response_format=completion_create_params.ResponseFormat(type="json_object")
)

print(completion.choices[0].message.model_dump_json())

BadRequestError: Error code: 400 - {'error': {'message': "'messages' must contain the word 'json' in some form, to use 'response_format' of type 'json_object'. (request id: 20240121122959735066473AEkDkcoT)", 'type': 'invalid_request_error', 'param': 'messages', 'code': None}}

In [1]:
# stream流式输出
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ],
    stream=True
)

for chunk in completion:
    print(chunk.choices[0].delta.model_dump_json())


{"content":"","function_call":null,"role":"assistant","tool_calls":null}
{"content":"Hello","function_call":null,"role":null,"tool_calls":null}
{"content":"!","function_call":null,"role":null,"tool_calls":null}
{"content":" How","function_call":null,"role":null,"tool_calls":null}
{"content":" can","function_call":null,"role":null,"tool_calls":null}
{"content":" I","function_call":null,"role":null,"tool_calls":null}
{"content":" assist","function_call":null,"role":null,"tool_calls":null}
{"content":" you","function_call":null,"role":null,"tool_calls":null}
{"content":" today","function_call":null,"role":null,"tool_calls":null}
{"content":"?","function_call":null,"role":null,"tool_calls":null}
{"content":null,"function_call":null,"role":null,"tool_calls":null}


In [27]:
# logit_bias参数
from openai import OpenAI
import tiktoken

enc = tiktoken.get_encoding("cl100k_base")
print(enc.encode("Hello"))

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Hello!"}
    ],
    logit_bias={enc.encode("Hello")[0]: -100}
)

print(completion.choices[0].message.model_dump_json())

[9906]
{"content":"Hi there! How can I assist you today?","role":"assistant","function_call":null,"tool_calls":null}


In [5]:
# logprobs参数
from openai import OpenAI

client = OpenAI()

completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=[
        {"role": "user", "content": "Hello!"}
    ],
    logprobs=True,
    top_logprobs=2
)

print(completion.choices[0].message.model_dump_json())
print(completion.choices[0].logprobs.model_dump_json())

{"content":"Hello! How can I assist you today?","role":"assistant","function_call":null,"tool_calls":null}
{"content":[{"token":"Hello","bytes":[72,101,108,108,111],"logprob":-0.21570958,"top_logprobs":[{"token":"Hello","bytes":[72,101,108,108,111],"logprob":-0.21570958},{"token":"Hi","bytes":[72,105],"logprob":-1.6720008}]},{"token":"!","bytes":[33],"logprob":-0.20926832,"top_logprobs":[{"token":"!","bytes":[33],"logprob":-0.20926832},{"token":" there","bytes":[32,116,104,101,114,101],"logprob":-1.6910303}]},{"token":" How","bytes":[32,72,111,119],"logprob":-0.00095136015,"top_logprobs":[{"token":" How","bytes":[32,72,111,119],"logprob":-0.00095136015},{"token":" Is","bytes":[32,73,115],"logprob":-7.526584}]},{"token":" can","bytes":[32,99,97,110],"logprob":-0.014271167,"top_logprobs":[{"token":" can","bytes":[32,99,97,110],"logprob":-0.014271167},{"token":" are","bytes":[32,97,114,101],"logprob":-4.673554}]},{"token":" I","bytes":[32,73],"logprob":-0.00001700133,"top_logprobs":[{"tok

In [17]:
from openai import OpenAI

client = OpenAI()

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    }
]
messages = [{"role": "user", "content": "What's the weather like in Boston today?"}]
completion = client.chat.completions.create(
    model="gpt-3.5-turbo-1106",
    messages=messages,
    tools=tools,
    tool_choice="auto"
)

print(completion.model_dump_json())

{"id":"chatcmpl-8a338gVY0y65AzAMrajrFvVHrXPVV","choices":[{"finish_reason":"tool_calls","index":0,"logprobs":null,"message":{"content":null,"role":"assistant","function_call":null,"tool_calls":[{"id":"call_yAjy5Ktcqn1unsqt9XgPRWku","function":{"arguments":"{\"location\":\"Boston, MA\",\"unit\":\"celsius\"}","name":"get_current_weather"},"type":"function"}]}}],"created":1703602222,"model":"gpt-3.5-turbo-1106","object":"chat.completion","system_fingerprint":"fp_772e8125bb","usage":{"completion_tokens":22,"prompt_tokens":83,"total_tokens":105}}


In [20]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
                },
            ],
        }
    ],
    max_tokens=300,
)

print(response.choices[0].model_dump_json())


{"finish_reason": "stop", "index": 0, "message": {"content": "This is a scenic image of a wooden boardwalk or pathway extending through a lush, green meadow or wetland area. The long grasses on either side of the boardwalk hint at a natural, possibly protected environment. The sky is partly cloudy with ample blue areas, suggesting it might be a pleasant day. Trees and shrubs are visible in the distance, possibly marking the end of this area or a transition to a different ecosystem. The scene is tranquil and invites walking or nature observation.", "role": "assistant", "function_call": null, "tool_calls": null}}
